# SVM: Version 5

## SVM with PCA and SIFT (Color Channels)

In [1]:
import re
import cv2
import os
import glob
import time
import mlflow
import random
import shutil
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

In [2]:
TRAIN_DIR = "../data/train"
CLASS_FOLDERS = ["Class A", "Class B", "Class C", "Class D"]
ML_FLOW_DIRECTORY = "SVM_Logs"

In [3]:
mlflow.set_tracking_uri(ML_FLOW_DIRECTORY)
mlflow.set_experiment("SVM_PCA_SIFT")

<Experiment: artifact_location='/Users/jacob/Code/Monkey-Business/SVM/SVM_Logs/950606938770948643', creation_time=1701979073386, experiment_id='950606938770948643', last_update_time=1701979073386, lifecycle_stage='active', name='SVM_PCA_SIFT', tags={}>

In [4]:
pca_components = [40 * 40, 45 * 45, 50 * 50]
C_values = [0.1, 1, 10]
kernel_types = ["linear", "rbf"]
gamma_values = ["scale", "auto"]

In [5]:
hyperparameter_combinations = list(
    itertools.product(pca_components, C_values, kernel_types, gamma_values)
)
print(
    f"Total number of hyperparameter combinations: {len(hyperparameter_combinations)}"
)

Total number of hyperparameter combinations: 36


In [6]:
pca_models_dict = {}

In [7]:
def train_with_params(pca_components, C, kernel, gamma, train_images, train_labels):
    # Check if PCA model for pca_components already exists
    if pca_components in pca_models_dict:
        pca = pca_models_dict[pca_components]
        print(f"Using existing PCA model for {pca_components} components")
    else:
        # Apply PCA
        pca = PCA(n_components=pca_components)
        pca.fit(train_images)
        pca_models_dict[pca_components] = pca

    # Transform images with PCA
    train_images_pca = pca.transform(train_images)

    with mlflow.start_run() as run:
        start_time = time.time()

        # Initialize SVM model
        svm_model = svm.SVC(
            C=C, kernel=kernel, gamma=gamma, class_weight="balanced", verbose=True
        )

        # Perform cross-validation
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scoring = {
            "f1_weighted": make_scorer(f1_score, average="weighted"),
            "accuracy": "accuracy",
            "precision": make_scorer(precision_score, average="weighted"),
            "recall": make_scorer(recall_score, average="weighted"),
        }
        cv_results = cross_validate(
            svm_model,
            train_images_pca,
            train_labels,
            cv=skf,
            scoring=scoring,
            verbose=2,
        )

        # Log hyperparameters and metrics
        mlflow.log_params(
            {"pca_components": pca_components, "C": C, "kernel": kernel, "gamma": gamma}
        )
        mlflow.log_metrics(
            {
                "mean_f1_weighted": cv_results["test_f1_weighted"].mean(),
                "mean_accuracy": cv_results["test_accuracy"].mean(),
                "mean_precision": cv_results["test_precision"].mean(),
                "mean_recall": cv_results["test_recall"].mean(),
            }
        )
        end_time = time.time()
        duration = end_time - start_time
        param_details = f"PCA={pca_components}, C={C}, kernel={kernel}, gamma={gamma}"
        duration_details = f"Training duration for {param_details}: {duration} seconds"
        print(duration_details)
        run_id = run.info.run_id
        return run_id

In [8]:
sift = cv2.SIFT_create()

In [9]:
def extract_sift_features(image_path, sift_model):
    image = cv2.imread(image_path)
    channels = cv2.split(image)

    all_descriptors = np.array([])

    for channel in channels:
        keypoints, descriptors = sift_model.detectAndCompute(channel, None)
        all_descriptors = np.concatenate(
            (all_descriptors, descriptors.flatten()), axis=0
        )
    return all_descriptors

In [10]:
def load_images_and_features(folder, sift_model):
    all_features = []
    labels = []
    max_features_per_image = 0

    for class_folder in CLASS_FOLDERS:
        start_time = time.time()
        class_path = os.path.join(folder, class_folder)
        image_files = glob.glob(os.path.join(class_path, "*.png"))

        for img_file in image_files:
            descriptors_per_channel = extract_sift_features(img_file, sift_model)
            max_features_per_image = max(max_features_per_image, len(descriptors_per_channel))
            all_features.append(descriptors_per_channel)
            labels.append(class_folder)

        end_time = time.time()
        duration = end_time - start_time
        print(f"Processing {class_folder} took {duration} seconds")

    # Use the maximum number of features found in any image for padding
    for i in range(len(all_features)):
        padded_descriptors = np.pad(all_features[i], (0, max_features_per_image - len(all_features[i])))
        all_features[i] = padded_descriptors

    return np.array(all_features), np.array(labels)

In [11]:
train_images, train_labels = load_images_and_features(TRAIN_DIR, sift)

Processing Class A took 17.063393115997314 seconds
Processing Class B took 9.76036524772644 seconds
Processing Class C took 6.936616897583008 seconds
Processing Class D took 8.25230622291565 seconds


In [12]:
print(
    f"Shape of train_images: {train_images.shape}, Shape of train_labels: {train_labels.shape}"
)

Shape of train_images: (2796, 129920), Shape of train_labels: (2796,)


In [ ]:
run_ids = []

for params in hyperparameter_combinations:
    pca_components, C, kernel, gamma = params
    run_id = train_with_params(
        pca_components, C, kernel, gamma, train_images, train_labels
    )
    run_ids.append(run_id)

In [18]:
run_metrics = {}
run_params = {}

for run_id in run_ids:
    client = mlflow.tracking.MlflowClient(ML_FLOW_DIRECTORY)
    run = client.get_run(run_id)
    metrics = run.data.metrics
    params = run.data.params
    run_metrics[run_id] = metrics
    run_params[run_id] = params

for run_id in run_ids:
    print(f"Run ID: {run_id}")
    print("Metrics:")
    metrics = run_metrics[run_id]
    for metric, value in metrics.items():
        print(f"\t{metric}: {value}")

    print("Parameters:")
    params = run_params[run_id]
    for param, value in params.items():
        print(f"\t{param}: {value}")

Run ID: 798a585a31424e7f97f43dac309df969
Metrics:
	mean_f1_weighted: 0.5603177736813261
	mean_accuracy: 0.5618741058655222
	mean_recall: 0.5618741058655222
	mean_precision: 0.5613189887179011
Parameters:
	gamma: scale
	pca_components: 1600
	C: 0.1
	kernel: linear
Run ID: 954ca0d54f304838bd32bbc7219cd6dc
Metrics:
	mean_f1_weighted: 0.5603177736813261
	mean_accuracy: 0.5618741058655222
	mean_recall: 0.5618741058655222
	mean_precision: 0.5613189887179011
Parameters:
	gamma: auto
	pca_components: 1600
	C: 0.1
	kernel: linear
Run ID: 67b1add7421d45068d8c3e839de3f5e8
Metrics:
	mean_f1_weighted: 0.4229527430076809
	mean_accuracy: 0.4620886981402003
	mean_recall: 0.4620886981402003
	mean_precision: 0.4333159501301751
Parameters:
	gamma: scale
	pca_components: 1600
	C: 0.1
	kernel: rbf
Run ID: d6e54f87cfb748478b1a381aa3be2f8e
Metrics:
	mean_f1_weighted: 0.08033054698280266
	mean_accuracy: 0.22103004291845493
	mean_recall: 0.22103004291845493
	mean_precision: 0.04913288143086076
Parameters:
	gam

In [19]:
best_run_id = max(run_metrics, key=lambda x: run_metrics[x]["mean_f1_weighted"])
best_run_params = run_params[best_run_id]
best_run_metrics = run_metrics[best_run_id]

print(f"\nBest Run ID: {best_run_id}")
print("Best Parameters:")
for key, value in best_run_params.items():
    print(f"\t{key}: {value}")

print("\nBest Run Metrics:")
for metric, value in best_run_metrics.items():
    print(f"\t{metric}: {value}")


Best Run ID: 4ada290a58a64dd39120c7dc1f665fd0
Best Parameters:
	gamma: scale
	pca_components: 1600
	C: 10
	kernel: rbf

Best Run Metrics:
	mean_f1_weighted: 0.5953484245539592
	mean_accuracy: 0.605865522174535
	mean_recall: 0.605865522174535
	mean_precision: 0.6057869960959049
